## ЛАБОРАТОРНАЯ РАБОТА №10.
## НАИВНЫЙ БАЙЕСОВСКИЙ КЛАССИФИКАТОР
#### по дисциплине «Математическая статистика»
Направление подготовки 01.03.02 Прикладная математика и информатика
Очной формы обучения

выполнил: студент группы Б9123-01.03.02ии

Иванов Матвей Олегович

принял: Деревягин Андрей Алексеевич

# Математическая часть

### Наивный Байесовский классификатор

Наивный Байесовский классификатор основан на теореме Байеса. Для нового объекта с признаками
$x = (x_1, x_2, \cdots, x_m)$ условная вероятность принадлежности к классу $y_k$ вычисляется как:
$$p_{Y|X}(y_k|x) = \frac{p_Y(y_k)*p_{X|Y}(x|y_k)}{p_X(x)}$$

Для вычисления совместной функции вероятности предположим, что признаки независимы в
совокупности. И так как знаменатель $p_X(x)$ одинаков для всех классов $y_k$ , он не влияет на результат
выбора наиболее вероятного класса. Поэтому для классификации достаточно максимизировать
числитель:
$$y = \argmax_{y_k} (\widehat{p}_Y(y_k) * \prod^{n}_{i=1} \widehat{p}_{X_i|Y(x_i|y_k)})$$

Можно использовать логарифмы, предполагая независимость признаков. Тогда итоговый вид
модели:
$$y = \argmax_{y_k} (\log(\widehat{p}_Y(y_k)) + \sum^{m}_{i=1} \log(\widehat{p}_{X_i|Y(x_i|y_k)}))$$

где $\widehat{p}_{Y_i}(y)$ - относительная частота значения $x у$ признака $Y_i$ . $\widehat{p}_{X_i|Y}(x|y)$ - относительная частота
значения $x у$ признака $X_i$ среди объектов класса $Y$.

### Оценка вероятностей

Для непрерывных признаков предполагается нормальное распределение, и вероятность заменяется
плотностью нормального распределения:
$$f_{X_i|Y}(x_i|y_k) = \frac{1}{\sqrt{2\pi}σ_{ik}}e^{-\frac{(x_i - µ_{ik})^2}{2σ^2_{ik}}}$$

где $µ_{ik}$ — среднее арифметическое, а $σ_{ik}$ — среднеквадратическое отклонение значений признака
$X_i$ для объектов класса $y_k$.

Для дискретных признаков $p_{X_i|Y}(x_i|y_k)$ оценка проводится как относительная частота значения
$x_i$ признака $X_i$ среди объектов класса $y_k$. Чтобы исключить нулевые вероятности, используется
сглаживание: к числителю и знаменателю добавляется 1.

# листинг кода

In [1]:
#необходимые импорты
import numpy as np #библиотека для работы с массивами
import scipy.stats as sps #пакет со статистикой для плотности вероятностей нормального распределения
from sklearn.metrics import accuracy_score #функция вычисления доли правильно предсказанных классов
from sklearn.model_selection import train_test_split #функция разбиения выборки на тренировочную и тестовую
from sklearn.naive_bayes import GaussianNB #библиотечный классификатор для непрерывных признаков
from sklearn.naive_bayes import MultinomialNB #библиотечный классификатор для дискретных признаков
from sklearn.datasets import load_iris, load_digits #датасет ирисы фишера

In [ ]:
class NaiveBias:
    def __init__(self, discrete_flags: list[bool]):
        self.discrete_flags: np.ndarray[bool] = np.array(discrete_flags)
        self.all_classes: np.ndarray[int]
        self.class_probs: np.ndarray[float]
        self.feature_params: dict[int, np.ndarray[dict[int, float] | tuple[float, float]]] = {}

    def fit(self, X: np.ndarray[np.ndarray[int]], y: np.ndarray[int]):
        self.all_classes = np.unique(y)
        n_features = X.shape[1]
        
        # Вычисляем априорные вероятности классов
        self.class_probs = np.array([np.mean(np.array(y == c, dtype=np.int8)) for c in self.all_classes])
        
        # Для каждого класса и каждого признака сохраняем параметры распределения
        for i, c in enumerate(self.all_classes):
            class_mask = np.array(y == c)
            class_data = X[class_mask]  # только те записи, который имеют y=c
            self.feature_params[c] = []
            
            for feature_i in range(n_features):
                feature_data = class_data[:, feature_i]  # забираем фичу
                is_discrete_feature = self.discrete_flags[feature_i]
                
                if is_discrete_feature:
                    # дискретные
                    values, counts = np.unique(feature_data, return_counts=True)
                    prob_dict = {v: c/len(feature_data) for v, c in zip(values, counts)}
                    self.feature_params[c].append(prob_dict)
                else:
                    # непрерывные
                    mean = np.mean(feature_data)
                    std = np.std(feature_data)
                    self.feature_params[c].append((mean, std))

    def predict(self, X):
        predictions = []
        for sample in X:
            class_scores = []

            for i, c in enumerate(self.all_classes):
                prob = self.class_probs[i]

                for feature in range(len(sample)):
                    if self.discrete_flags[feature]:
                        # дискретные
                        prob_dict = self.feature_params[c][feature]
                        value = sample[feature]
                        prob *= prob_dict.get(value, 1e-10)
                    else:
                        # непрерывные
                        mean, std = self.feature_params[c][feature]
                        prob *= sps.norm.pdf(sample[feature], mean, std)

                class_scores.append(prob)

            predicted_class = self.all_classes[np.argmax(class_scores)]
            predictions.append(predicted_class)
        
        return np.array(predictions)

In [ ]:
class NaiveBiasLog:
    def __init__(self, discrete_flags: list[bool]):
        self.discrete_flags: np.ndarray[bool] = np.array(discrete_flags)
        self.all_classes: np.ndarray[int]
        self.class_probs: np.ndarray[float]
        self.feature_params: dict[int, np.ndarray[dict[int, float] | tuple[float, float]]] = {}

    def fit(self, X: np.ndarray[np.ndarray[int]], y: np.ndarray[int]):
        self.all_classes = np.unique(y)
        n_features = X.shape[1]
        
        # Вычисляем априорные вероятности классов
        self.class_probs = np.array([np.mean(np.array(y == c, dtype=np.int8)) for c in self.all_classes])
        
        # Для каждого класса и каждого признака сохраняем параметры распределения
        for i, c in enumerate(self.all_classes):
            class_mask = np.array(y == c)
            class_data = X[class_mask]  # только те записи, который имеют y=c
            self.feature_params[c] = []
            
            for feature_i in range(n_features):
                feature_data = class_data[:, feature_i]  # забираем фичу
                is_discrete_feature = self.discrete_flags[feature_i]
                
                if is_discrete_feature:
                    # дискретные
                    values, counts = np.unique(feature_data, return_counts=True)
                    prob_dict = {v: c/len(feature_data) for v, c in zip(values, counts)}
                    self.feature_params[c].append(prob_dict)
                else:
                    # непрерывные
                    mean = np.mean(feature_data)
                    std = np.std(feature_data)
                    self.feature_params[c].append((mean, std))

    def predict(self, X):
        predictions = []
        for sample in X:
            class_scores = []

            for i, c in enumerate(self.all_classes):
                prob = np.log(self.class_probs[i])

                for feature in range(len(sample)):
                    if self.discrete_flags[feature]:
                        # дискретные
                        prob_dict = self.feature_params[c][feature]
                        value = sample[feature]
                        prob += np.log(prob_dict.get(value, 1e-10))
                    else:
                        # непрерывные
                        mean, std = self.feature_params[c][feature]
                        prob += np.log(sps.norm.pdf(sample[feature], mean, std))

                class_scores.append(prob)

            predicted_class = self.all_classes[np.argmax(class_scores)]
            predictions.append(predicted_class)
        
        return np.array(predictions)

In [3]:
# проверка работы с непрерывными значениями

iris = load_iris()
X = iris.data
y = iris.target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

discrete_flags = [False, False, False, False]

our_model = NaiveBias(discrete_flags)
our_model.fit(X_train, y_train)

our_predictions = our_model.predict(X_test)
our_accuracy = accuracy_score(y_test, our_predictions)
print(f"Точность нашей реализации: {our_accuracy:.4f}")

lib_model = GaussianNB()
lib_model.fit(X_train, y_train)
lib_predictions = lib_model.predict(X_test)
lib_accuracy = accuracy_score(y_test, lib_predictions)
print(f"Точность GaussianNB: {lib_accuracy:.4f}")

Точность нашей реализации: 0.9600
Точность GaussianNB: 0.9600


In [4]:
# проверка работы с дискретными значениями

digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

discrete_flags = [True for _ in range(64)]


our_model = NaiveBias(discrete_flags)
our_model.fit(X_train, y_train)

our_predictions = our_model.predict(X_test)
our_accuracy = accuracy_score(y_test, our_predictions)
print(f"Точность нашей реализации: {our_accuracy:.4f}")

lib_model = MultinomialNB()
lib_model.fit(X_train, y_train)
lib_predictions = lib_model.predict(X_test)
lib_accuracy = accuracy_score(y_test, lib_predictions)
print(f"Точность MultinomialNB: {lib_accuracy:.4f}")

Точность нашей реализации: 0.7880
Точность MultinomialNB: 0.8881
